In [1]:
import numpy as np
import collections
import os

path = os.path.abspath(os.getcwd())

from prep.derivatives import Preprocess_derivatives

from src.supplementary import Define_Derivatives
from src.term import normalize_ts, Term
from src.trainer import Equation_Trainer

Задаём функцию, позволяющую оценивать значений слагаемых уравнения. Это исполнение использует тензоры значений токенов на сетке и через поэлементное умножение тензоры, получает значения слагаемого. Далее, тензор преобразуется в вектор для использования в качестве признака в регрессии. 

In [2]:
def derivative_evaluator(term, normalize, eval_params):
    
    '''
    
    Example of the evaluator of token values, appropriate for case of derivatives with pre-calculated values, defined on grid, that take form of tensors
    
    Parameters
    ----------
    term : term.Term, or numpy.ndarray
        Object for term of the equation, or its gene, for which the evaluation is done; necessary for the evaluation.
    eval_params : dict
        Dictionary, containing parameters of the evaluator: in this example, they are 
        'token matrices' : list/numpy.martix of token (derivatives) values on the grid, 'parameter_indexes' : dictionary of orders of token parameters during the encoding. 
        In simplest case of only power parameter: 'parameter_indexes':{'power':0}.
    
    Returns
    ----------
    value : numpy.ndarray
        Vector of the evaluation of the token values, that shall be used as target, or feature during the LASSO regression.
        
    '''
    
    assert 'token_matrices' in eval_params and 'parameter_indexes' in eval_params
    if type(term) == Term:
        term = term.gene
    token_matrices = eval_params['token_matrices']
    value = np.copy(token_matrices[0])
    for var_idx in np.arange(term.shape[0]):
        power = (term[var_idx + eval_params['parameter_indexes']['power']])
        value *= eval_params['token_matrices'][int(var_idx / (float(eval_params['parameter_indexes']['power']+1)))] ** int(power)
    if normalize:
        value = normalize_ts(value)
    value = value.reshape(np.prod(value.shape))
    return value    

Проводим препроцессинг данных и вычисляем значения производных на сетке. Вычисления довольно трудоёмкие => распараллелены, но всё равно могут занять относительно много времени, особенно если считать на ПК. Результаты препроцессинга сохраняем в отдельный файл, чтобы использовать при повторных запусках основного алгоритма. Это экономит время! Для примера используется решение волнового уравнения с белым шумом.
<br>
В этом примере мы рассмотрим задачу поиска уравнения по синтетическим данным, полученным из решения волнового уравнения: 
<br>
$\frac{\partial^2 u}{\partial t^2} = \frac{\partial^2 u}{\partial x_1^2} + \frac{\partial^2 u}{\partial x_2^2}$,
<br>
которое отражает эволюцию некоторой величины $u$ в двумерной области. Данные для этого эксперимента можно взять по ссылке: https://drive.google.com/open?id=1joW0zTwkSGLJVpyWxDqoSMzTvRItX24J

In [3]:
op_file_name =  path + '/Preprocessing/Derivatives.npy'
filename =  path + '/Preprocessing/wave_HP.npy'
poolsize = 4

if 'npy' in filename:
    field = np.load(filename)
else:
    shape = (201, 201, 201)
    field = np.loadtxt(filename)
    field = field.reshape(shape)
field = np.transpose(field, (2, 0, 1))
Preprocess_derivatives(field, op_file_name, mp_poolsize=poolsize)

Executing on grid with uniform nodes:
Start: 2020-03-03 18:33:33.448178 ; Finish: 2020-03-03 19:19:32.216328
Preprocessing runtime: 0:45:58.768150


Загружаем значения в узлах сетки для исходной функции и её производных; из них формируем тензор для дальнейшего использования. Также задаём границы области, по которому будет вычисляться уравнение (в примере - обрезаем начало и конец временного ряда + по 15 элементов с каждой границы, чтобы использовать более "качественные" производные)

In [4]:
u_initial = np.load('Preprocessing/Wave_HP/wave_HP.npy') # Пропишите путь к файлу с исходным полем 
u_initial = np.transpose(u_initial, (2, 0, 1)) 
print(u_initial.shape)

derivatives = np.load('Preprocessing/Wave_HP/Derivatives.npy') # Пропишите путь к файлу с производными 
variables = np.ones((2 + derivatives.shape[1], ) + u_initial.shape)
variables[1, :] = u_initial
for i_outer in range(0, derivatives.shape[1]):
    variables[i_outer+2] = derivatives[:, i_outer].reshape(variables[i_outer+2].shape) 
    
skipped_elems = 15 
timeslice = (skipped_elems, -skipped_elems)

variables = variables[:, timeslice[0]:timeslice[1], skipped_elems:-skipped_elems, skipped_elems:-skipped_elems]     

(101, 101, 101)


Получаем названия токенов для производных, используя функцию **Define_Derivatives()**. Она получает названия токенов в порядке: 1, u, $\frac{\partial u}{\partial x_1}$, $\frac{\partial^2 u}{\partial x_1^2}$, ... , $\frac{\partial u}{\partial x_2}$, $\frac{\partial^2 u}{\partial x_2^2}$, ...
<br>
Далее зададим параметры для токенов: в этом примере единственным параметром является степень токена, используемого в слагаемом. Например, если 'power' = 2 для токена $\frac{\partial u}{\partial x_1}$, то в слагаемом будет $ (\frac{\partial u}{\partial x_1})^2 $. Также зададим слагаемые, которые будут в каждом уравнении: константу и исходную функцию. 

In [5]:
token_names = Define_Derivatives(u_initial.ndim, max_order = 2)
print(token_names)

token_parameters = collections.OrderedDict([('power', (0, 3))])
basic_terms = [{'1':{'power':1}},
               {'1':{'power':1},  'u':{'power':1}}]   

('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')


Задаём объект для обучения уравнения:

In [6]:
Trainer = Equation_Trainer(tokens = token_names, token_params = token_parameters, 
                           evaluator = derivative_evaluator, 
                           evaluator_params = {'token_matrices':variables, 'parameter_indexes':{'power':0}},
                           basic_terms = basic_terms)

Так как мы не знаем, какой параметр $\alpha$ - коэффициент регуляризации в LASSO, позволяет получить правильную структуру уравнения, мы проводим запуск по сетке из гиперпараметров модели. Сетка в примере строится только по одному параметру ($\alpha$), но в общем виде допустимо задавать сетки сразу по нескольким. Для остальные гиперпараметров модели задаём соответствующие значения. В случае, если каждый параметр задаётся одним значением, то их допустимо подавать сразу в метод Train.  

In [7]:
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_param_mutation', 'r_mutation', 
                         'mut_chance', 'pop_size', 'eq_len', 'max_factors'), 
                        ((0.01, 0.16, 4), 0.2, 0.6, 0.8, 0.5, 0.8, 20, 6, 2))

Запускаем обучение и получаем искомое уравнение в символьной форме

In [8]:
Trainer.Train(epochs = 50)

Using parameters from grid
Achieved best fitness: 0.0050783488205190215 with alpha = 0.01
Discovered equation:
- {  d^2u/dx1^2 : {'power': 1.0}} + -0.0001963014668564108 * {  u : {'power': 1.0}} + 1.003430266231036 * {  d^2u/dx2^2 : {'power': 1.0}} + 0.000465301995916656 * {  du/dx1 : {'power': 1.0}} + 1.003430266231037 * {  d^2u/dx3^2 : {'power': 1.0}} = 0
Achieved best fitness: 0.004485088661452463 with alpha = 0.06
Discovered equation:
- {  d^2u/dx1^2 : {'power': 1.0}} + 1.0013594947213769 * {  d^2u/dx3^2 : {'power': 1.0}} + -0.00020581754275531233 * {  u : {'power': 1.0}} + 1.001359494721376 * {  d^2u/dx2^2 : {'power': 1.0}} = 0
Achieved best fitness: 0.0038249581919527357 with alpha = 0.10999999999999999
Discovered equation:
- {  d^2u/dx1^2 : {'power': 1.0}} + 1.0935985825740213 * {  d^2u/dx2^2 : {'power': 1.0}} + 1.0935985825740218 * {  d^2u/dx3^2 : {'power': 1.0}} = 0
Achieved best fitness: 0.003611457117138705 with alpha = 0.16
Discovered equation:
- {  d^2u/dx1^2 : {'power': 1

Для основной части исследуемого интервала $\alpha$ мы получаем структуры уравнения в формате
<br>
$\frac{\partial^2 u}{\partial t^2} = a_1 \frac{\partial^2 u}{\partial x_1^2} + a_2 \frac{\partial^2 u}{\partial x_2^2}$, 
<br>
где коэффициенты $a_1$ и $a_2$ отличаются от исходных, равных 1, в силу погрешностей, в большей степени связанных с численным решением исходного дифференциального уравнения, а также с погрешностями вычисления производных. 
<br>
При слишком низких значениях $\alpha$ вектор коэффициентов уравнения недостаточно разрежен, и в уравнении присутствуют лишние слагаемые, хотя и обладают незначительными весами. 
<br>

In [10]:
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_param_mutation', 'r_mutation', 
                         'mut_chance', 'pop_size', 'eq_len', 'max_factors'), 
                        ((0.3, 0.4, 2), 0.2, 0.6, 0.8, 0.5, 0.8, 20, 6, 2))

In [11]:
Trainer.Train(epochs = 50)

Using parameters from grid
Achieved best fitness: 0.00227045231823162 with alpha = 0.3
Discovered equation:
- {  du/dx1 : {'power': 1.0}  d^2u/dx1^2 : {'power': 1.0}} + -0.0004731879609027912 * {  u : {'power': 1.0}} + -0.6650816595841387 * {  d^2u/dx2^2 : {'power': 1.0}} + -0.6650816595841381 * {  d^2u/dx3^2 : {'power': 1.0}} = 0
Achieved best fitness: 0.002505082208075159 with alpha = 0.4
Discovered equation:
- {  d^2u/dx1^2 : {'power': 1.0}} + 1.0935985825740213 * {  d^2u/dx2^2 : {'power': 1.0}} + 1.0935985825740218 * {  d^2u/dx3^2 : {'power': 1.0}} = 0


Можно отметить, что в силу стохастического характера построения уравнений для описания системы и 
протекающей эволюции, возможны случаи, когда алгоритм не сходится к лучшему решениюб а остается в некотором локальном оптимуме функции приспособленности. Так в случае уравнения для $\alpha = 0.3$, получается неправильное уравнение.
<br>
Индикатором этого является сравнительно низкое значение функции приспособленности, отражающей "качество" уравнени, которое превышается "правильными" структурами даже при больших значениях коэффициента регуляризации. Детали задания фитнес-функции представлены в разделе wiki на github-странице проекта и в соответстующих статьях. 
<br>
Для избежания таких локальных оптимумов, алгоритм рекомендуется запускать несколько раз с одними и теми же данными и выбором значений, соответсвующих максимуму функции приспособленности. 

In [7]:
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_param_mutation', 'r_mutation', 
                         'mut_chance', 'pop_size', 'eq_len', 'max_factors'), 
                        ((2, 3, 2), 0.2, 0.6, 0.8, 0.5, 0.8, 20, 6, 2))

In [8]:
Trainer.Train(epochs = 50)

Using parameters from grid
Achieved best fitness: 0.0006686093848979459 with alpha = 2.0
Discovered equation:
- {  du/dx1 : {'power': 1.0}} = 0
Achieved best fitness: 0.0006686093848979459 with alpha = 3.0
Discovered equation:
- {  du/dx1 : {'power': 1.0}} = 0


При дальнейшем увеличении коэффициента регуляризации, уравнение меняет свой вид: теряются значимые слагаемые и принимает вид
<br>
$\frac{\partial^2 u}{\partial t^2} = 0$: